In [16]:
# train a 3 layer MLP on mnist
import torch
import torch.nn as nn
import torchvision 
from torchvision import transforms
import os
import numpy as np 
import tqdm

model = nn.Sequential(
    nn.Linear(784, 150, bias=False),
    nn.ReLU(),
    nn.Linear(150, 200, bias=False),
    nn.ReLU(),
    nn.Linear(200, 10, bias=False),
)

train_set = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        torch.flatten
    ])
)
test_set = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        torch.flatten
    ])
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=128,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_set,
    batch_size=128,
    shuffle=True
)

loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim,
    mode='max',
    factor=0.1,
    patience=2,
    min_lr = 1e-4
)


epochs = 12
eval_every = 100 # batches 

best_model = None

with torch.autocast("cuda"):
    for epoch in tqdm.tqdm(range(epochs)):
        
        train_loss = 0
        eval_loss = 0
        train_acc = 0
        eval_acc = 0
        best_acc = 0
        
        # train
        for i, (x, y) in enumerate(train_loader):
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            
            acc = (y_pred.argmax(dim=1) == y).float().mean()
            train_acc += acc 
            train_loss += loss.item()
            
            # eval
            if i % eval_every == 0:
                for x, y in test_loader:
                    model.eval()
                    y_pred = model(x)
                    loss = loss_fn(y_pred, y)
                    
                    eval_loss += loss.item()
                    acc = (y_pred.argmax(dim=1) == y).float().mean()
                    eval_acc += acc    
                    
                print(f"Epoch {epoch}: train acc: {train_acc / eval_every}, train loss: {train_loss / eval_every} \
                        , eval acc: {eval_acc / len(test_loader)}, eval loss: {eval_loss / len(test_loader)}\
                        lr: {optim.param_groups[0]['lr']}"
                    )
                
                if eval_acc / len(test_loader) > best_acc:
                    best_acc = eval_acc / len(test_loader)
                    best_model = model.state_dict()
                    
                train_loss = 0
                eval_loss = 0
                train_acc = 0
                eval_acc = 0
        
        scheduler.step(eval_acc / len(test_loader))

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0: train acc: 0.0010937500046566129, train loss: 0.023133807182312012, eval acc: 0.12480221688747406, eval loss: 2.2896044616457782
Epoch 0: train acc: 0.7679687738418579, train loss: 0.8164816942811012, eval acc: 0.8830102682113647, eval loss: 0.3744132918647573
Epoch 0: train acc: 0.8978906273841858, train loss: 0.33041078358888626, eval acc: 0.9067444801330566, eval loss: 0.2938633547930778
Epoch 0: train acc: 0.924609363079071, train loss: 0.2633488894253969, eval acc: 0.934335470199585, eval loss: 0.22176922697432433
Epoch 0: train acc: 0.9344531297683716, train loss: 0.2194523583352566, eval acc: 0.9376977682113647, eval loss: 0.19800268471995486


 10%|█         | 1/10 [00:17<02:36, 17.43s/it]

Epoch 1: train acc: 0.009140624664723873, train loss: 0.0026094257831573487, eval acc: 0.9476858973503113, eval loss: 0.17335808729823632
Epoch 1: train acc: 0.9498437643051147, train loss: 0.1726352559775114, eval acc: 0.9516416192054749, eval loss: 0.1560080115317921
Epoch 1: train acc: 0.9547656178474426, train loss: 0.15803983636200428, eval acc: 0.9575751423835754, eval loss: 0.13968276142885414
Epoch 1: train acc: 0.9541406035423279, train loss: 0.14909979697316886, eval acc: 0.9599485993385315, eval loss: 0.13456800778078126
Epoch 1: train acc: 0.9606249928474426, train loss: 0.1312691617757082, eval acc: 0.9629153609275818, eval loss: 0.12041272510644756


 20%|██        | 2/10 [00:34<02:16, 17.06s/it]

...

 80%|████████  | 8/10 [02:15<00:33, 16.88s/it]

Epoch 8: train acc: 0.009921874850988388, train loss: 0.0004186992347240448, eval acc: 0.9784414768218994, eval loss: 0.06945385421107558
Epoch 8: train acc: 0.9878125190734863, train loss: 0.046494239810854196, eval acc: 0.978342592716217, eval loss: 0.0695319443092316
Epoch 8: train acc: 0.9884374737739563, train loss: 0.042786774151027204, eval acc: 0.9781447649002075, eval loss: 0.06943445870228394
Epoch 8: train acc: 0.9891406297683716, train loss: 0.04184523501433432, eval acc: 0.9782436490058899, eval loss: 0.06955708134212071
Epoch 8: train acc: 0.9884374737739563, train loss: 0.043266087006777525, eval acc: 0.9769580960273743, eval loss: 0.07128854925873913


 90%|█████████ | 9/10 [02:32<00:16, 16.90s/it]

Epoch 9: train acc: 0.009921874850988388, train loss: 0.00022818226367235183, eval acc: 0.9784414768218994, eval loss: 0.06999730705460415
Epoch 9: train acc: 0.9885937571525574, train loss: 0.044662206079810855, eval acc: 0.9782436490058899, eval loss: 0.06939372750547348
Epoch 9: train acc: 0.9897656440734863, train loss: 0.044999213563278316, eval acc: 0.9785403609275818, eval loss: 0.06980168135671676
Epoch 9: train acc: 0.9882031083106995, train loss: 0.04265894245356321, eval acc: 0.9784414768218994, eval loss: 0.06950056836999292
Epoch 9: train acc: 0.9880468845367432, train loss: 0.04303899850696325, eval acc: 0.977749228477478, eval loss: 0.07096755187486924


100%|██████████| 10/10 [03:55<00:00, 23.52s/it]


In [28]:
# save the first 5000 images in the test set as numpy array
test_images = []
test_labels = []
for i, (x, y) in enumerate(test_loader):
    test_images.append(x)
    test_labels.append(y)
    if i == 5000:
        break
test_images = torch.cat(test_images).numpy()
test_labels = torch.cat(test_labels).numpy()
np.save("./data/test_images.npy", test_images)
np.save("./data/test_labels.npy", test_labels)

# save model weights as numpy array
for name, param in best_model.items():
    np.save(f"./data/linear_{name}.npy", param.T.detach().cpu().numpy()) # W @ X -> X @ W.T
    
np.load("./data/linear_0.weight.npy").shape


(784, 150)